### Imports

In [68]:
import pydub
from pydub import AudioSegment
import os 
from pydub.playback import play
from pydub import AudioSegment
from pydub.utils import make_chunks
import random 
from random import choice
os.environ["PATH"] += os.pathsep + '/usr/local/bin'

### Path Setup

In [69]:
# provide the name of the marine mammal that is desired to be
mammal_name = "HumpbackWhale"

# the path to the raw Watkins Audio Samples 
path_to_raw_data = f"../marine_data/{mammal_name}/"

# path to new directory to store lengthened (30s) audio
training_data_path = f"../training_data/{mammal_name}/"


### Function Declarations

In [70]:
def get_wavs_from_directory(dirpath):
    ''' extracts all files in the given directory
    '''
    
    playlist = []
    
    for file in os.listdir(dirpath):
        if file not in (".ipynb_checkpoints"):
            sound = AudioSegment.from_file(f"{dirpath}{file}") # f"{dirpath}{file}"
        
        # check length for 30 seconds in milliseconds
            while len(sound) < 30000:
                sound = sound * 2
        
            playlist.append(sound)
    
    
    return playlist

In [71]:
def save_new_audio(playlist, fish, path):
    '''make 30 second chuncks for each wav file.
    '''
    
    chunk_len = 30000
    os.mkdir(training_data_path)
    
    for idx, wav in enumerate(playlist):
        audio = make_chunks(wav, chunk_len)
        audio[0].export(f"{path}{fish}{idx}.wav", format="wav")

In [72]:
def get_wavs_from_training(dirpath= "../", fish="Walrus"):
    '''get path to training data and augment the audio
    '''
    
    aug_playlist = []
    aug_training_path = os.path.join(dirpath, '..', 'aug_training/')
    os.makedirs(aug_training_path, exist_ok = True) 
    print(f'saving files to {aug_training_path}')
    
    for file in os.listdir(dirpath):
        try:
            if file not in (".ipynb_checkpoints"):
                sound = AudioSegment.from_file(f"{dirpath}{file}")
                aug_playlist.append(sound)
        except IsADirectoryError as e:
            continue
            
    for idx, sound in enumerate(aug_playlist):
        octaves = random.choice([0.1667, -0.1667]) # randomly choose +/- 2 semitones
        dB = random.choice([3, -3]) # randomly choose +/- 3 dB
        
        new_sample_rate = int(sound.frame_rate * (2.0 ** octaves))
        new_sound = sound._spawn(sound.raw_data, overrides={'frame_rate': new_sample_rate})
        
        # grab the first 15 and last 15 and concatenate
        seconds = 15 * 1000
            
        first = new_sound[:seconds]
        last = new_sound[-seconds+1:] # add the - to get the last 15 sec
            
        aug_samp = first + last
        aug_samp = aug_samp + dB # increase/decrease volume 
        
        # export augmented audio into its appropriate class
        aug_samp.export(f"{aug_training_path}{fish}_aug_{idx}.wav", format="wav") # switch path b/t path or dirpath2

    print(f"saved {len(aug_playlist)} augmented files")

### Generate Training Data

1. Extend each audio track to 30 seconds

In [73]:
playlist = get_wavs_from_directory(path_to_raw_data)
playlist_len = len(playlist)
print(f'process {playlist_len} number of audio samples')

process 604 number of audio samples


2. Store processed audio samples into a new directory

In [74]:
save_new_audio(playlist, mammal_name, training_data_path)

### Augment Training Data
1. duplicate audio files
2. Augment this half for each class
3. Save augmented audio back in the appropriate class in the training data set


In [75]:
# store augmented files back into the original training directories
get_wavs_from_training(training_data_path, mammal_name)

saving files to ../training_data/HumpbackWhale/aug_training/
saved 604 augmented files
